<a href="https://colab.research.google.com/github/flammavitae/seasonal01/blob/main/POC_Main_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: from a given list of stock symbols, if CSV files exist load them. Otherwise, download closing prices for the last 20 years and save the prices as CSV files. For the loaded prices, combine into a single dataframe and remove dates where prices do not exist for all symbols. calculate percentage the differences in prices from one day to the next and create a separate dataframe for the logs of the differences. Calculate sharpe ratio, standard deviations, maximum drawdown and profit for each symbol. Calculate efficient frontier and optimum portfolio with maximum sharpe.

import pandas as pd
import numpy as np
import yfinance as yf
import os
from datetime import date, timedelta
from scipy.optimize import minimize

# Globals
data_dir = 'Data'


def download_and_process_stock_data(stock_symbols, start_date='2004-01-01', end_date=date.today().strftime('%Y-%m-%d')):
    """
    Downloads or loads stock data, calculates daily returns, logs of returns, Sharpe ratio, etc.
    """
    # Create a directory for storing CSV files
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    all_data = []
    short_symbols = []
    for symbol in stock_symbols:
        csv_file = f'{data_dir}/{symbol}.csv'

        if os.path.exists(csv_file):
            print(f"Loading data from CSV: {symbol}.csv")
            try:
              data = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
            except pd.errors.EmptyDataError:
              print(f"Error: {symbol}.csv is empty. Downloading...")
              data = yf.download(symbol, start=start_date, end=end_date)
              data.to_csv(csv_file)
            except pd.errors.ParserError:
              print(f"Error: {symbol}.csv parsing error. Downloading...")
              data = yf.download(symbol, start=start_date, end=end_date)
              data.to_csv(csv_file)
        else:
            print(f"Downloading data for {symbol}...")
            data = yf.download(symbol, start=start_date, end=end_date)
            data.to_csv(csv_file)

        all_data.append(data['Close'])
        short_symbols.append('short_' + symbol)

    # Combine data into a single dataframe
    combined_data = pd.concat(all_data, axis=1)

    # Short
    all_data = all_data * -1
    combined_data = pd.concat(all_data, axis=1)
    stock_symbols = stock_symbols.append(short_symbols)
    combined_data.columns = stock_symbols
    print(combined_data)

    # Remove dates with missing values
    combined_data = combined_data.dropna()

    # Calculate daily percentage change
    daily_returns = combined_data.pct_change().dropna()

    # Calculate logs of daily returns
    log_returns = np.log(1+daily_returns)

    # ... Rest of the code as before ...

    # Calculate Sharpe ratio, std dev, max drawdown, etc.
    results = {}
    for symbol in stock_symbols:
      results[symbol] = calculate_metrics(combined_data[symbol])

    # Efficient Frontier and Optimal portfolio (Illustrative example, may need to be modified based on your needs)
    # Example: assuming risk-free rate is 0.02
    risk_free_rate = 0.02
    calculate_efficient_frontier(daily_returns, risk_free_rate)

    return combined_data, daily_returns, log_returns, results


def calculate_metrics(price_series):
  """
  Calculate metrics like Sharpe ratio, standard deviation, max drawdown, and profit.
  """
  daily_returns = price_series.pct_change().dropna()
  sharpe = (daily_returns.mean() - 0.02) / daily_returns.std() * np.sqrt(252)  # Annualized Sharpe ratio
  std_dev = daily_returns.std() * np.sqrt(252)  # Annualized std dev
  max_drawdown = calculate_max_drawdown(price_series)
  profit = (price_series[-1] - price_series[0]) / price_series[0]  # total profit

  return {"Sharpe Ratio": sharpe, "Standard Deviation": std_dev, "Max Drawdown": max_drawdown, "Total Profit": profit}

def calculate_max_drawdown(price_series):
  # Find the peak and trough for each drawdown
  peak = price_series.expanding(min_periods=1).max()
  drawdown = (price_series - peak) / peak
  max_drawdown = drawdown.min()
  return max_drawdown

def calculate_efficient_frontier(returns, risk_free_rate):
  # Illustrative Example - replace this with your own efficient frontier calculation method
  print("Calculating efficient frontier (example implementation)...")
  # ... code to calculate efficient frontier ...
  print("Efficient Frontier Calculated.")



In [4]:
# Example usage:
stock_symbols = ['AAPL', 'MSFT', 'GOOG']
combined_data, daily_returns, log_returns, metrics = download_and_process_stock_data(stock_symbols)

print("\nCombined Stock Data:\n", combined_data.head())
print("\nDaily Returns:\n", daily_returns.head())
print("\nLog of Daily Returns:\n", log_returns.head())
print("\nMetrics", metrics)

Loading data from CSV: AAPL.csv


ValueError: 'Date' is not in list

In [81]:
# prompt: based on the contenbt file AAPL.csv, rewrite the reading of the csv to return only the closing price data

import pandas as pd
import numpy as np
import yfinance as yf
import os
from datetime import date, timedelta


def download_and_process_stock_data(stock_symbols, start_date='2004-01-01', end_date=date.today().strftime('%Y-%m-%d')):
    """
    Downloads or loads stock data, calculates daily returns, logs of returns, Sharpe ratio, etc.
    """
    # Create a directory for storing CSV files
    if not os.path.exists('stock_data'):
        os.makedirs('stock_data')

    all_data = []
    short_symbols = []
    for symbol in stock_symbols:
        csv_file = f'stock_data/{symbol}.csv'

        if os.path.exists(csv_file):
            print(f"Loading data from CSV: {symbol}.csv")
            try:
              # Read only the 'Close' column
              data = pd.read_csv(csv_file)
              # Remove the ticker row (assuming it's the first row)
              # No information was provided about the ticker row, so it will be assumed that the first row is the ticker row.
              data = data.iloc[2:]

              # Rename the 'Price' column to 'Date'
              data = data.rename(columns={'Price': 'Date'})
              #data = data[['Date', 'Close']]
              data['Close'] = np.float64(data['Close'])

              # Convert 'Date' column to datetime objects
              data['Date'] = pd.to_datetime(data['Date'])

              # Set 'Date' as the index
              data = data.set_index('Date')

            except pd.errors.EmptyDataError:
              print(f"Error: {symbol}.csv is empty. Downloading...")
              data = yf.download(symbol, start=start_date, end=end_date)
              data.to_csv(csv_file)
            except pd.errors.ParserError:
              print(f"Error: {symbol}.csv parsing error. Downloading...")
              data = yf.download(symbol, start=start_date, end=end_date)
              data.to_csv(csv_file)
        else:
            print(f"Downloading data for {symbol}...")
            data = yf.download(symbol, start=start_date, end=end_date)
            data.to_csv(csv_file)
        short_symbols.append('short_' + symbol)

        all_data.append(data['Close'])

    # Combine data into a single dataframe
    combined_data = pd.concat(all_data, axis=1)
    combined_data.columns = stock_symbols

    # # Short
    short_data = pd.DataFrame(all_data * -1)
    print(short_symbols)
    #short_data.columns = short_symbols
    print(short_data.columns)
    combined_data = pd.concat([combined_data, short_data], axis=1)
    #combined_data = combined_data.join(short_data)

    print(combined_data)

    # Remove dates with missing values
    combined_data = combined_data.dropna()
    print(combined_data)
    # Calculate daily percentage change
    daily_returns = combined_data.pct_change(1).dropna()

    # Calculate logs of daily returns
    log_returns = np.log(1+daily_returns)

    # ... Rest of the code as before ...

    # Calculate Sharpe ratio, std dev, max drawdown, etc.
    results = {}
    for symbol in stock_symbols:
      results[symbol] = calculate_metrics(combined_data[symbol])

    # Efficient Frontier and Optimal portfolio (Illustrative example, may need to be modified based on your needs)
    # Example: assuming risk-free rate is 0.02
    risk_free_rate = 0.02
    calculate_efficient_frontier(daily_returns, risk_free_rate)

    return combined_data, daily_returns, log_returns, results
    #return combined_data


def calculate_metrics(price_series):
  """
  Calculate metrics like Sharpe ratio, standard deviation, max drawdown, and profit.
  """
  daily_returns = price_series.pct_change().dropna()
  sharpe = (daily_returns.mean() - 0.02) / daily_returns.std() * np.sqrt(252)  # Annualized Sharpe ratio
  std_dev = daily_returns.std() * np.sqrt(252)  # Annualized std dev
  max_drawdown = calculate_max_drawdown(price_series)
  profit = (price_series[-1] - price_series[0]) / price_series[0]  # total profit

  return {"Sharpe Ratio": sharpe, "Standard Deviation": std_dev, "Max Drawdown": max_drawdown, "Total Profit": profit}

def calculate_max_drawdown(price_series):
  # Find the peak and trough for each drawdown
  peak = price_series.expanding(min_periods=1).max()
  drawdown = (price_series - peak) / peak
  max_drawdown = drawdown.min()
  return max_drawdown

def calculate_efficient_frontier(returns, risk_free_rate):
  # Illustrative Example - replace this with your own efficient frontier calculation method
  print("Calculating efficient frontier (example implementation)...")
  # ... code to calculate efficient frontier ...
  print("Efficient Frontier Calculated.")


# Example usage:
stock_symbols = ['AAPL', 'MSFT', 'GOOG']
combined_data, daily_returns, log_returns, metrics = download_and_process_stock_data(stock_symbols)
#combined_data = download_and_process_stock_data(stock_symbols)

print("\nCombined Stock Data:\n", combined_data.head())
print("\nDaily Returns:\n", daily_returns.head())
print("\nLog of Daily Returns:\n", log_returns.head())
print("\nMetrics", metrics)

Loading data from CSV: AAPL.csv
Loading data from CSV: MSFT.csv
Loading data from CSV: GOOG.csv
['short_AAPL', 'short_MSFT', 'short_GOOG']
RangeIndex(start=0, stop=0, step=1)
                  AAPL        MSFT        GOOG
Date                                          
2004-01-02    0.320548   17.041073         NaN
2004-01-05    0.333954   17.469440         NaN
2004-01-06    0.332749   17.531515         NaN
2004-01-07    0.340281   17.512890         NaN
2004-01-08    0.351880   17.481848         NaN
...                ...         ...         ...
2025-01-17  229.979996  429.029999  197.550003
2025-01-21  222.639999  428.500000  199.630005
2025-01-22  223.830002  446.200012  200.029999
2025-01-23  223.660004  446.709991  199.580002
2025-01-24  222.779999  444.059998  201.899994

[5300 rows x 3 columns]
                  AAPL        MSFT        GOOG
Date                                          
2004-08-19    0.462595   16.836203    2.490185
2004-08-20    0.463951   16.885881    2.687981
2

<ipython-input-81-c30e4d28274c>:106: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  profit = (price_series[-1] - price_series[0]) / price_series[0]  # total profit


In [72]:
# Short
short_data = combined_data  * -1
print(short_data)
combined_data = pd.concat(short_data, axis=1)
#stock_symbols = stock_symbols.append(short_symbols)



                  AAPL        MSFT        GOOG
Date                                          
2004-08-19   -0.462595  -16.836203   -2.490185
2004-08-20   -0.463951  -16.885881   -2.687981
2004-08-23   -0.468169  -16.960588   -2.715032
2004-08-24   -0.481274  -16.960588   -2.602608
2004-08-25   -0.497844  -17.153606   -2.630652
...                ...         ...         ...
2025-01-17 -229.979996 -429.029999 -197.550003
2025-01-21 -222.639999 -428.500000 -199.630005
2025-01-22 -223.830002 -446.200012 -200.029999
2025-01-23 -223.660004 -446.709991 -199.580002
2025-01-24 -222.779999 -444.059998 -201.899994

[5142 rows x 3 columns]


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [55]:
df = pd.DataFrame({"B": [0, 6, 2, 5, np.nan, 4]})

In [59]:
df.expanding(3).mean()

,B
0,NaN
1,NaN
2,2.666667
3,3.250000
4,3.250000
5,3.400000
